<a href="https://colab.research.google.com/github/Girija-Tandon/Tech-Kisan/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv("/content/greenhouse_data.csv")

In [51]:
df.head(5)

,Temperature (°C),Humidity (%),Moisture (%),Soil Type,Crop Type,Carbon Emission (ppm)
0,39.07,86.58,24.14,Loamy,Barley,280.22
1,27.09,44.44,34.65,Loamy,Barley,281.43
2,35.15,37.29,23.78,Clayey,Corn,204.04
3,28.76,41.85,20.73,Clayey,Wheat,374.99
4,16.09,83.22,24.83,Peaty,Soybean,553.09


In [73]:
df['Crop Type'].unique()

array([0, 1, 4, 3, 2])

In [52]:
df.isnull().sum()

,0
Temperature (°C),0
Humidity (%),0
Moisture (%),0
Soil Type,0
Crop Type,0
Carbon Emission (ppm),0


In [53]:
df.describe()

,Temperature (°C),Humidity (%),Moisture (%),Carbon Emission (ppm)
count,300.000000,300.000000,300.000000,300.000000
mean,27.413667,60.637567,24.687633,399.407100
std,7.281472,17.286701,8.518443,114.428192
min,15.280000,30.190000,10.050000,200.810000
25%,20.830000,45.112500,18.360000,298.962500
50%,27.605000,61.520000,24.575000,401.640000
75%,33.505000,75.735000,31.375000,502.055000
max,39.920000,89.900000,39.860000,599.820000


In [54]:
df.dtypes

,0
Temperature (°C),float64
Humidity (%),float64
Moisture (%),float64
Soil Type,object
Crop Type,object
Carbon Emission (ppm),float64


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score


In [56]:
label_encoders = {}
for col in ["Soil Type", "Crop Type"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [57]:
features = ["Temperature (°C)", "Humidity (%)", "Moisture (%)", "Soil Type", "Crop Type"]
target_regression = ["Carbon Emission (ppm)"]  # Predict environmental impact
target_classification = "Greenhouse Action"

In [58]:
df[target_classification] = np.where(df["Moisture (%)"] < 40, "Increase Watering", "Maintain")
df[target_classification] = np.where(df["Temperature (°C)"] > 35, "Turn On Cooling", df[target_classification])


In [59]:
# Split Data into Train & Test Sets
X_train, X_test, y_train_reg, y_test_reg = train_test_split(df[features], df[target_regression], test_size=0.2, random_state=42)
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(df[features], df[target_classification], test_size=0.2, random_state=42)


In [60]:
#  Scale Numeric Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [61]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train_scaled, y_train_reg.values.ravel())

RandomForestRegressor(random_state=42)

In [62]:
# Train Classification Model (Recommend Greenhouse Action)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_cls, y_train_cls)

RandomForestClassifier(random_state=42)

In [63]:
# Evaluate Model Performance
y_pred_reg = regressor.predict(X_test_scaled)
reg_mae = mean_absolute_error(y_test_reg, y_pred_reg)
print(f"Regression MAE (Carbon Emission Prediction): {reg_mae:.2f}")

Regression MAE (Carbon Emission Prediction): 102.33


In [64]:

y_pred_cls = classifier.predict(X_test_cls)
cls_acc = accuracy_score(y_test_cls, y_pred_cls)
print(f"Classification Accuracy (Greenhouse Action): {cls_acc:.2%}")

Classification Accuracy (Greenhouse Action): 96.67%


In [71]:
def predict_greenhouse_action(temp, humidity, moisture, soil_type, crop_type):
    # Encode categorical inputs
    soil_encoded = label_encoders["Soil Type"].transform([soil_type])[0]
    crop_encoded = label_encoders["Crop Type"].transform([crop_type])[0]

    # Use the exact feature names from training
    input_data = pd.DataFrame([[temp, humidity, moisture, soil_encoded, crop_encoded]],
                              columns=["Temperature (°C)", "Humidity (%)", "Moisture (%)", "Soil Type", "Crop Type"])

    # Apply scaling
    input_scaled = scaler.transform(input_data)

    # Make Predictions
    carbon_pred = regressor.predict(input_scaled)[0]
    action_pred = classifier.predict(input_data)[0]

    return {
        "Predicted Carbon Emission (ppm)": round(carbon_pred, 2),
        "Recommended Greenhouse Action": action_pred
    }

# Example Usage
print(predict_greenhouse_action(25, 60, 30, "Loamy", "Wheat"))


{'Predicted Carbon Emission (ppm)': np.float64(440.77), 'Recommended Greenhouse Action': 'Increase Watering'}
